In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import sys
# Standard Library Imports
import math
from typing import NoReturn

# Third-Party Imports
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchsummary import summary
from torchvision import transforms

In [ ]:
#!pip install grad-cam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 23.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.4.8-py3-none-any.whl size=38247 sha256=442ed2e7cd2db5b2f124c5db1bb81978dc321d5401fdf63f4627060dce753de1
  Stored in directory: /root/.cache/pip/wheels/f8/04/36/94ff3c8a4215826a21946b34c01180817e606989fdf53f7cd6
Successfully built grad-cam


In [ ]:
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

In [ ]:
import warnings
warnings.simplefilter("ignore")
warnings.warn("deprecated", DeprecationWarning)
warnings.filterwarnings( "ignore", module = "matplotlib\..*" )

In [ ]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
def allot_device(random_seed_value):
    if torch.cuda.is_available():
        device = "cuda"
        torch.manual_seed(random_seed_value)
    else:
        device = "cpu"
    return device

In [ ]:
class Plots:
    def __init__(self,mis_classify_details=None ,num_images=None, loaded_data=None):
        self.num_images = num_images
        self.loaded_data = loaded_data
        self.mis_classify = mis_classify_details


    def plot_images(self):
        batch_data, batch_label = next(iter(self.loaded_data))
        fig = plt.figure()
        if self.num_images % 2 != 0:
            self.num_images -= 1
        self.num_rows = self.num_images // 4

        fig = plt.figure(figsize=(15, 7))
        counter = 0
        for i in range(self.num_images):
            sub = fig.add_subplot(self.num_rows, 4, i + 1)
            im2display = (np.squeeze(batch_data[i].permute(2, 1, 0)))
            sub.imshow(im2display.cpu().numpy())
            sub.set_title(batch_label[i].item())
            sub.axis('off')

        plt.tight_layout()
        plt.axis('off')
        plt.show()

    def mis_classified_gradcam(self):
        model, testloader, device, images_needed = self.mis_classify
        storing_images = []
        storing_predicted_labels = []
        storing_target_labels = []

        target_layers = [model.layer4[-1]]

        # Construct the CAM object once, and then re-use it on many images:
        cam = GradCAM(model=model, target_layers=target_layers, use_cuda=True)
        targets = [ClassifierOutputTarget(281)]

        if images_needed == None:
            images_needed = random.choice([10, 20])
        with torch.no_grad():
            model.eval()
            for data, target in testloader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
                for idx in range(len(pred)):
                    if pred[idx] != target[idx]:
                        # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
                        grayscale_cam = cam(input_tensor=data[idx], targets=targets)
                        visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)
                        #storing_images.append(data[idx])
                        storing_images.append(visualization)
                        storing_predicted_labels.append(pred[idx])
                        storing_target_labels.append(target[idx])

        fig = plt.figure(figsize=(20, 14))

        if images_needed % 2 != 0:
            images_needed -= 1  # It becomes even so plotting would be good.

        num_rows = 0
        plots_per_row = 0
        if images_needed <= 10:
            num_rows = 5
            plots_per_row = images_needed // num_rows
        elif 22 > images_needed > 10:
            num_rows = 4
            plots_per_row = images_needed // num_rows
        elif images_needed > 20:
            num_rows = 10
            plots_per_row = images_needed // num_rows

        for i in range(images_needed):
            sub = fig.add_subplot(num_rows, plots_per_row, i + 1)
            im2display = (np.squeeze(storing_images[i].permute(2, 1, 0)))
            sub.imshow(im2display.cpu().numpy())
            sub.set_title(
                f"Predicted as: {classes[storing_predicted_labels[i]]} \n But, Actual is: {classes[storing_target_labels[i]]}")
        plt.tight_layout()
        plt.show()

    def mis_classified(self):
        model, testloader, device, images_needed = self.mis_classify
        storing_images = []
        storing_predicted_labels = []
        storing_target_labels = []
        if images_needed == None:
            images_needed = random.choice([10, 20])
        with torch.no_grad():
            model.eval()
            for data, target in testloader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
                for idx in range(len(pred)):
                    if pred[idx] != target[idx]:
                        storing_images.append(data[idx])
                        storing_predicted_labels.append(pred[idx])
                        storing_target_labels.append(target[idx])

        fig = plt.figure(figsize=(20, 14))

        if images_needed % 2 != 0:
            images_needed -= 1  # It becomes even so plotting would be good.

        num_rows = 0
        plots_per_row = 0
        if images_needed <= 10:
            num_rows = 5
            plots_per_row = images_needed // num_rows
        elif 22 > images_needed > 10:
            num_rows = 4
            plots_per_row = images_needed // num_rows
        elif images_needed > 20:
            num_rows = 10
            plots_per_row = images_needed // num_rows

        for i in range(images_needed):
            sub = fig.add_subplot(num_rows, plots_per_row, i + 1)
            im2display = (np.squeeze(storing_images[i].permute(2, 1, 0)))
            sub.imshow(im2display.cpu().numpy())
            sub.set_title(
                f"Predicted as: {classes[storing_predicted_labels[i]]} \n But, Actual is: {classes[storing_target_labels[i]]}")
        plt.tight_layout()
        plt.show()

In [ ]:
def plot_metrics(metrics):
    if metrics is None:
        print("Please provide the metric values, unable to view them!")
        sys.exit(0)
    else:
        train_losses, train_acc, test_losses, test_acc = metrics
        fig, axs = plt.subplots(2, 2, figsize=(15, 10))
        axs[0, 0].plot(train_losses)
        axs[0, 0].set_title("Training Loss")
        axs[1, 0].plot(train_acc)
        axs[1, 0].set_title("Training Accuracy")
        axs[0, 1].plot(test_losses)
        axs[0, 1].set_title("Test Loss")
        axs[1, 1].plot(test_acc)
        axs[1, 1].set_title("Test Accuracy")

In [ ]:
def printTrainTest_LossAcc(train_losses, train_acc, test_losses,test_acc):
  fig, axs = plt.subplots(2,2,figsize=(15,10))
  axs[0, 0].plot(train_losses)
  axs[0, 0].set_title("Training Loss")
  axs[1, 0].plot(train_acc)
  axs[1, 0].set_title("Training Accuracy")
  axs[0, 1].plot(test_losses)
  axs[0, 1].set_title("Test Loss")
  axs[1, 1].plot(test_acc)
  axs[1, 1].set_title("Test Accuracy")

In [ ]:
def get_misclassified_data(model, device, test_loader):
    """
    Function to run the model on test set and return misclassified images
    :param model: Network Architecture
    :param device: CPU/GPU
    :param test_loader: DataLoader for test set
    """
    # Prepare the model for evaluation i.e. drop the dropout layer
    model.eval()

    # List to store misclassified Images
    misclassified_data = []

    # Reset the gradients
    with torch.no_grad():
        # Extract images, labels in a batch
        for data, target in test_loader:

            # Migrate the data to the device
            data, target = data.to(device), target.to(device)

            # Extract single image, label from the batch
            for image, label in zip(data, target):

                # Add batch dimension to the image
                image = image.unsqueeze(0)

                # Get the model prediction on the image
                output = model(image)

                # Convert the output from one-hot encoding to a value
                pred = output.argmax(dim=1, keepdim=True)

                # If prediction is incorrect, append the data
                if pred != label:
                    misclassified_data.append((image, label, pred))
    return misclassified_data

In [ ]:
def display_gradcam_output(data: list,
                           classes: list[str],
                           inv_normalize: transforms.Normalize,
                           model: 'DL Model',
                           target_layers: list['model_layer'],
                           targets=None,
                           number_of_samples: int = 10,
                           transparency: float = 0.60):
    """
    Function to visualize GradCam output on the data
    :param data: List[Tuple(image, label)]
    :param classes: Name of classes in the dataset
    :param inv_normalize: Mean and Standard deviation values of the dataset
    :param model: Model architecture
    :param target_layers: Layers on which GradCam should be executed
    :param targets: Classes to be focused on for GradCam
    :param number_of_samples: Number of images to print
    :param transparency: Weight of Normal image when mixed with activations
    """
    # Plot configuration
    fig = plt.figure(figsize=(10, 10))
    x_count = 5
    y_count = 1 if number_of_samples <= 5 else math.floor(number_of_samples / x_count)

    # Create an object for GradCam
    cam = GradCAM(model=model, target_layers=target_layers, use_cuda=True)

    # Iterate over number of specified images
    for i in range(number_of_samples):
        plt.subplot(y_count, x_count, i + 1)
        input_tensor = data[i][0]

        # Get the activations of the layer for the images
        grayscale_cam = cam(input_tensor=input_tensor, targets=targets)
        grayscale_cam = grayscale_cam[0, :]

        # Get back the original image
        img = input_tensor.squeeze(0).to('cpu')
        img = inv_normalize(img)
        rgb_img = np.transpose(img, (1, 2, 0))
        rgb_img = rgb_img.numpy()

        # Mix the activations on the original image
        visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True, image_weight=transparency)

        # Display the images on the plot
        plt.imshow(visualization)
        plt.title(r"Correct: " + classes[data[i][1].item()] + '\n' + 'Output: ' + classes[data[i][2].item()])
        plt.xticks([])
        plt.yticks([])